In [81]:
import swat
import os

from sasctl import Session
from sasctl.tasks import register_model, publish_model
from sasctl.services import microanalytic_score as mas

In [82]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'

In [85]:
s = swat.CAS('frasepviya34.aws.sas.com', 8777, username='viyademo01', password='demopw', protocol='https')

In [88]:
s = swat.CAS('frasepviya34.aws.sas.com', 443, username='viyademo01', password='demopw', protocol='https')

SWATError: [('', '', '')]

In [70]:
s.loadactionset('decisionTree')

NOTE: Added action set 'decisionTree'.


[actionset]

 'decisionTree'

+ Elapsed: 0.00145s, user: 0.000845s, sys: 0.000585s, mem: 0.211mb

In [71]:
tbl = s.CASTable('iris')
if not tbl.tableexists().exists:
    indata = s.upload_file('http://support.sas.com/documentation/onlinedoc/viya/exampledatasets/iris.csv', casout=tbl)

NOTE: Cloud Analytic Services made the uploaded file available as table IRIS in caslib CASUSER(viyademo01).
NOTE: The table IRIS has been created in caslib CASUSER(viyademo01) from binary data uploaded to Cloud Analytic Services.


In [77]:
tbl = s.CASTable('iris')
tbl.decisiontree.gbtreetrain(target='Species',
                             inputs=['Sepal Length', 'Sepal Width',
                                     'Petal Length', 'Petal Width'],
                             savestate='gradboost_astore')

NOTE: Wrote 644251 bytes to the savestate file gradboost_astore.


,Descr,Value
0,Number of Trees,150.0
1,Distribution,3.0
2,Learning Rate,0.1
3,Subsampling Rate,0.5
4,Number of Selected Variables (M),4.0
5,Number of Bins,50.0
6,Number of Variables,4.0
7,Max Number of Tree Nodes,15.0
8,Min Number of Tree Nodes,3.0
9,Max Number of Branches,2.0


In [78]:
astore = s.CASTable('gradboost_astore')

In [79]:
with Session(s):
    model = register_model(astore, 'Gradient Boosting', 'Iris',force=True)
    module = publish_model(model, 'maslocal')
    response = mas.execute_module_step(module, 'score',
                                       SepalLength=5.1,
                                       SepalWidth=3.5,
                                       PetalLength=1.4,
                                       PetalWidth=0.2)

NOTE: Added action set 'astore'.


SWATCASActionError: The action was not successful.

In [80]:
s.close()